In [ ]:
import os
import csv
import cv2
from concurrent.futures import ThreadPoolExecutor
import argparse

ap = argparse.ArgumentParser()
ap.add_argument("-v", "--VIDEO_DIR", required=True, help="path to the directory containing the videos")
ap.add_argument("-o", "--OUTPUT_DIR", required=True, help="path to the output directory")
ap.add_argument("-n", "--MAX_WORKERS", default=os.cpu_count(), type=int,
                help="number of processes to use for processing the videos")
args = vars(ap.parse_args())

if not os.path.exists(args["OUTPUT_DIR"]):
    os.makedirs(args["OUTPUT_DIR"])

def process_video(video_file):
    # Load the video file
    cap = cv2.VideoCapture(video_file)
    frame_count = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))

    # Create output directory for the video
    video_name = os.path.splitext(os.path.basename(video_file))[0]
    video_output_dir = os.path.join(args["OUTPUT_DIR"], video_name)
    os.makedirs(video_output_dir, exist_ok=True)

    # Create output data file for the video
    data_file = os.path.join(video_output_dir, f"{video_name}.csv")

    # Open CSV file for writing
    with open(data_file, mode='w') as csv_file:
        fieldnames = ['frame', 'person', 'x', 'y', 'w', 'h']
        writer = csv.DictWriter(csv_file, fieldnames=fieldnames)
        writer.writeheader()

        # Process each frame in the video
        for i in range(frame_count):
            # Read the frame
            ret, frame = cap.read()

            if not ret:
                break

            # Apply person detection algorithm to the frame
            # You can use any pre-trained object detection model in OpenCV here
            # Here's an example using the Haar cascades classifier for detecting pedestrians
            classifier = cv2.CascadeClassifier(cv2.data.haarcascades + "haarcascade_fullbody.xml")
            gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
            detections = classifier.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=2)

            # If people are detected in the frame, save the frame and its data
            if len(detections) > 0:
                frame_file = os.path.join(video_output_dir, f"{i:04d}.jpg")
                cv2.imwrite(frame_file, frame)

                # Save the frame data in the output data file
                for j, (x, y, w, h) in enumerate(detections):
                    writer.writerow({
                        "frame": i,
                        "person": j,
                        "x": x,
                        "y": y,
                        "w": w,
                        "h": h
                    })

    cap.release()

def process_videos():
    video_files = [os.path.join(args["VIDEO_DIR"], f) for f in os.listdir(args["VIDEO_DIR"]) if f.endswith(".mp4")]

    # Create a thread pool executor with a maximum number of worker threads
    with ThreadPoolExecutor(max_workers=args["MAX_WORKERS"]) as executor:
        # Submit the video processing tasks to the thread pool
        futures = [executor.submit(process_video, video_file) for video_file in video_files]

        # Wait for all tasks to complete
        for future in futures:
            future.result()

process_videos()
